In [3]:
# pip install numpy scipy scikit-image matplotlib


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import json
import os

In [2]:
class CustomDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        with open(annotations_file) as f:
            self.annotations = json.load(f)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_info = self.annotations[idx]
        img_path = os.path.join(self.img_dir, img_info['/home/yasumasa-tezuka/data_intern/data/images_expansion'])
        image = Image.open(img_path).convert("RGB")
        boxes = torch.tensor(img_info['boxes'])
        labels = torch.tensor(img_info['labels'])
        target = {'boxes': boxes, 'labels': labels}

        if self.transform:
            image = self.transform(image)

        return image, target

In [5]:
# 画像の前処理の定義
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor()
])

# データセットの作成
dataset = CustomDataset(
    annotations_file='/home/yasumasa-tezuka/data_intern/data/notes.json',
    img_dir='/home/yasumasa-tezuka/data_intern/data//images',
    transform=transform
)

# データローダーの作成
data_loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)


In [8]:
import torch
from torchvision.models.detection import ssd300_vgg16

model = ssd300_vgg16(pretrained=True)

# カスタムデータセットのクラス数に合わせてモデルのクラス分類ヘッドを変更
num_classes = 1  # ここにクラス数を設定（背景クラスも含む）
for i in range(len(model.head.classification_head.module_list)):
    in_channels = model.head.classification_head.module_list[i].in_channels
    new_conv = torch.nn.Conv2d(in_channels, num_classes * 4, kernel_size=3, padding=1)
    model.head.classification_head.module_list[i] = new_conv


In [10]:
# 損失関数とオプティマイザーの設定
# from utils.loss import MultiBoxLoss
# criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos = 3, device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# デバイスの設定
device = torch.device('cuda' if torch.cuda.is_available() else 'gpu')
model.to(device)

# 訓練ループ
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {losses.item()}')


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import sys
sys.path.append('/path/to/the/directory/containing/utils')

from utils.loss import MultiBoxLoss


In [ ]:
import os

def find_file_in_directory(directory, filename):
    for root, dirs, files in os.walk(directory):
        if filename in files:
            return os.path.join(root, filename)
    return None

# ここで 'MultiBoxLoss.py' は探しているファイル名に置き換えてください
file_path = find_file_in_directory('プロジェクトのルートディレクトリ', 'MultiBoxLoss.py')
print(file_path)
